In [1]:
from transporter.transporter.none_hybrid.NoneHybrid import GA
from transporter.transporter.GA_refactoring.GA_refactoring import HGA
from transporter.transporter.multi_start.MultiStart import MultiStart, get_dir_path
from transporter.data.create_data.FileManager import FileManager
from transporter.data.create_data.Graph import Graph
from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pickle
import os

font_path = r'C:\Windows\Fonts\gulim.ttc'
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)
plt.rcParams['axes.unicode_minus'] = False

In [3]:
block = 300
cluster = 2

ga_params = {
    'POPULATION_SIZE': 100,
    'GENERATION_SIZE': 400,
    'ELITISM_RATE': 0.05,
    'MUTATION_RATE': 0.1,
    'SELECTION_METHOD': 'selection2',
}


precondition = {
    'START_TIME': 9,  # 전제
    'FINISH_TIME': 18,  # 전제
    'LOAD_REST_TIME': 0.2,  # 전제
    'BLOCKS': block,  # 전제
}

data_path = os.path.join(get_dir_path("transporter"), "data")


In [3]:
filemanager = FileManager()

# 그래프 파일 받아오기
node_file_path = os.path.join(data_path, "nodes_and_blocks", "cluster", "simply_mapping", str(cluster), f"node.csv")
graph = Graph(node_file_path)
shortest_path_dict = graph.get_shortest_path_dict()

# 블록 받아오기
block_path = os.path.join(data_path, "nodes_and_blocks", "cluster", "simply_mapping", str(cluster), f"block{block}.csv")
block_container = filemanager.load_block_data(block_path, BLOCK_NUM=precondition['BLOCKS'])

# 트랜스포터 받아오기
transporter_path = os.path.join(data_path, 'transporters', 'transporter.csv')
transporter_container = filemanager.load_transporters(transporter_path)


In [4]:
hga = HGA(transporter_container, block_container, graph, ga_params, precondition)
ga = GA(transporter_container, block_container, graph, ga_params, precondition)
ms = MultiStart(transporter_container, block_container, graph, ga_params, precondition)

In [ ]:
result_dict = {'HGA': [], "GA": [], "MS": []}
for i in range(5):
    result_dict['MS'].append(ms.run_GA())
    result_dict['HGA'].append(hga.run_GA())
    result_dict['GA'].append(ga.run_GA())

100%|██████████| 400/400 [02:26<00:00,  2.73it/s, Multi Start Best Transporter Count=80]


MultiStart best: 80, best_fitness_value: 5239.7043606812085


 73%|███████▎  | 291/400 [06:24<02:16,  1.25s/it, HGA Best Transporter Count=46]

In [6]:
def make_dataframe(result_dict, type):
    df = pd.DataFrame(columns=['best_fitness', 'best_distance'])
    for result in result_dict[type]:

        best_distance = result['best_distance']
        best_fitness = result['best_fitness']
        best_time_span = result['best_time_span']
        best_tp_count = result['work_tp_count'][-1]
        series = pd.Series({'best_fitness': best_fitness, 'best_distance': best_distance,
                            'work_tp_count': best_tp_count, 'best_time_span': best_time_span})
        df = df.append(series, ignore_index=True)
    return df


def make_dataset(df, type, tag):
    dataset_df = df[tag]
    dataset_df = pd.concat([dataset_df, pd.Series([type] * len(dataset_df))], axis=1)
    dataset_df.columns = [tag, 'group']
    return dataset_df

hga_df = make_dataframe(result_dict, "HGA")
ga_df = make_dataframe(result_dict, "GA")
ms_df = make_dataframe(result_dict, "MS")

hga_dataset = make_dataset(hga_df, "HGA", "best_time_span")
ga_dataset = make_dataset(ga_df, "GA", "best_time_span")
ms_dataset = make_dataset(ms_df, "MS", "best_time_span")

concatenated_df = pd.concat([hga_dataset, ga_dataset, ms_dataset], axis=0)

In [16]:
# result4(ga), result5(multistart)

plt.figure(figsize=(6, 6), dpi=400)
sns.boxplot(x='group', y='best_time_span', data=concatenated_df)
plt.title('알고리즘 비교  (ga_gen: 500, ms_loop: 1000)')

# 각 그룹의 최고점과 최저점을 계산합니다.
groups = concatenated_df.groupby('group')['best_time_span']
min_points = round(groups.min(),3)
max_points = round(groups.max(), 3)

plt.show()


In [17]:
with open('../transporter/GA_refactoring/pickle_data/Hybrid_GA_cluster21111.pkl', 'rb') as f:
    result = pickle.load(f)

ValueError: Could not interpret input 'work_tp_count'

<Figure size 2400x2400 with 0 Axes>